# 依赖

In [1]:
!pip install influxdb-client

You should consider upgrading via the 'd:\python\python37\python.exe -m pip install --upgrade pip' command.


# 环境

In [9]:
from pprint import pprint

from datetime import datetime

# https://docs.influxdata.com/influxdb/v2.0/api-guide/client-libraries/python/
# > https://influxdb-client.readthedocs.io/en/stable/api.html
# https://github.com/influxdata/influxdb-client-python
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate a Token from the "Tokens Tab" in the UI
token = "UdkcUg0sIsVqCuOV6RcQsPedRV5kFIE7pp5w7BoFtGRgc5DmcMqvElHF3gc0zGoJYt0gloVH-_eCax6VXKnOhA=="
org = "zhoujiagen"
bucket = "zhoujiagen"

# 初始化客户端

In [3]:
client = InfluxDBClient(url="http://localhost:8086", token=token)

# 写入数据

## Option 1: Use InfluxDB Line Protocol to write data

In [4]:
write_api = client.write_api(write_options=SYNCHRONOUS)

data = "mem,host=host1 used_percent=23.43234543"
write_api.write(bucket, org, data)

## Option 2: Use a Data Point to write data

In [7]:
point = Point("mem").tag("host",
                         "host1").field("used_percent", 23.43234543).time(
                             datetime.utcnow(), WritePrecision.NS)

write_api.write(bucket, org, point)

## Option 3: Use a Batch Sequence to write data

In [8]:
sequence = [
    "mem,host=host1 used_percent=23.43234543",
    "mem,host=host1 available_percent=15.856523"
]
write_api.write(bucket, org, sequence)

# 查询数据

In [21]:
query = f"""from(bucket: "{bucket}") 
|> range(start: -1h)
|> filter(fn: (r) => r["_measurement"] == "mem")
|> filter(fn:(r) => r._field == "available_percent" or r._field == "used_percent" )
""".format(bucket=bucket)
tables = client.query_api().query(query, org=org)

# https://github.com/influxdata/influxdb-client-python/blob/master/influxdb_client/client/flux_table.py
if tables is not None and len(tables) > 0:
    for table in tables:
        for record in table.records:
            print(record.get_time(), record.get_field(), record.get_value())


2021-08-12 09:19:34.465501+00:00 available_percent 15.856523
2021-08-12 09:17:33.423349+00:00 used_percent 23.43234543
2021-08-12 09:19:01.085829+00:00 used_percent 23.43234543
2021-08-12 09:19:34.465501+00:00 used_percent 23.43234543
